In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from scripts.utils.data import LeaflictionData

batch_size = 128

data = LeaflictionData(
    original_dir="/Users/a.villa.massone/Code/42/42_Leaffliction/images/Apple",
    force_preproc=False,
    test_split=0.2,
    val_split=0.2,
    batch_size=batch_size,
)


[2025-09-17 18:05:34] [INFO] scripts.utils.data: Initializing LeaflictionData
[2025-09-17 18:05:34] [INFO] scripts.utils.data: Datasets are ready. Skipping preprocessing
[2025-09-17 18:05:34] [INFO] scripts.utils.data: Successfully loaded '/Users/a.villa.massone/Code/42/42_Leaffliction/images_augmented/Apple'
[2025-09-17 18:05:34] [INFO] scripts.utils.data: Successfully loaded '/Users/a.villa.massone/Code/42/42_Leaffliction/images_augmented/Apple_train'
[2025-09-17 18:05:34] [INFO] scripts.utils.data: Successfully loaded '/Users/a.villa.massone/Code/42/42_Leaffliction/images_augmented/Apple_val'
[2025-09-17 18:05:34] [INFO] scripts.utils.data: Successfully loaded '/Users/a.villa.massone/Code/42/42_Leaffliction/images_augmented/Apple_test'
[2025-09-17 18:05:34] [INFO] scripts.utils.data: Successfully loaded '/Users/a.villa.massone/Code/42/42_Leaffliction/images_augmented/Apple_train_raw'
[2025-09-17 18:05:34] [INFO] scripts.utils.data: Data loaders available : dict_keys(['train', 'test'

In [2]:
from scripts.utils.model import LeaflictionCNN
import torch

device = ("cuda" if torch.cuda.is_available()
          else "mps" if torch.backends.mps.is_available()
          else "cpu")
print(f"Using {device} device")

model = LeaflictionCNN(num_classes=4, device=device).to(device)
print(model)

[2025-09-17 18:05:34] [INFO] scripts.utils.model: Initializing LeaflictionCNN


Using mps device
LeaflictionCNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): AdaptiveAvgPool2d(output_size=1)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=32, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=4, bias=True)
  )
)


In [3]:
def train_one_epoch(model, dataloader, loss_fn, optimizer):
    model.train()
    device = next(model.parameters()).device

    running_loss = 0.0
    running_acc = 0.0
    seen = 0

    for batch_idx, (X, y) in enumerate(tqdm(dataloader, total=len(dataloader)), start=1):
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(X)
        loss = loss_fn(logits, y)
        loss.backward()
        optimizer.step()

        # Log progress
        bsz = X.size(0)
        running_loss += loss.item() * bsz
        running_acc  += (logits.argmax(1) == y).float().sum().item()
        seen += bsz

        # Log progress every 50 batches of 64 images (every 3200 images)
        # if batch_idx % 50 == 0 or batch_idx == len(dataloader):
        #     print(f"[{batch_idx:>4d}/{len(dataloader)}] "
        #           f"train_loss: {running_loss/seen:.4f}  train_acc: {running_acc/seen:.4f}")



In [ ]:
def evaluate(model, dataloader, text, loss_fn=None):
    """
    Evaluates the model on a dataloader.
    If loss_fn is provided, also returns/prints average loss.
    Returns: (avg_loss_or_None, avg_acc)
    """
    model.eval()
    device = next(model.parameters()).device

    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    y_true, y_pred = [], []

    results = {}

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            logits = model(X)

            y_true += y.tolist()
            y_pred += logits.argmax(1).cpu().tolist()

            if loss_fn is not None:
                total_loss += loss_fn(logits, y).item() * X.size(0)

            total_correct += (logits.argmax(1) == y).sum().item()
            total_samples += X.size(0)

    accuracy = total_correct / max(1, total_samples)
    if loss_fn is not None:
        loss = total_loss / max(1, total_samples)
        print(f"{text}:/t{text}_loss={loss:.4f}  {text}_acc={accuracy:.4f}")
    else:
        print(f"{text}:/t{text}_acc={accuracy:.4f}")

    results['accuracy'] = accuracy
    results['loss'] = loss
    results['y_true'] = y_true
    results['y_pred'] = y_pred

    return results

In [5]:
# Dataset size (number of samples)
N = len(data.loaders['train'].dataset)

# Batch size requested (None if using batch_sampler)
B = data.loaders['train'].batch_size

N, B, len(data.loaders['train']), len(data.loaders['val'])


(14189, 128, 111, 4)

In [6]:
from tqdm import tqdm
from torch import nn
from pathlib import Path
import matplotlib.pyplot as plt

# --- history container ---
history = {
    "train_loss": [],
    "train_acc":  [],
    "val_loss":   [],
    "val_acc":    [],
}

learning_rate = 1e-3
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate) # Adam
epochs = 10

best_acc, patience, bad = 0.0, 7, 0
best_state = None

print("Starting training of LeaflictionCNN")
for epoch in range(1, epochs + 1):

    print(f"-------------------------------\nEpoch {epoch}")
    train_one_epoch(model=model, dataloader=data.loaders['train'], loss_fn=loss, optimizer=optimizer)

    train_results = evaluate(model=model, dataloader=data.loaders['train'], loss_fn=loss, text='train')
    val_results = evaluate(model=model, dataloader=data.loaders['val'], loss_fn=loss, text='val')

    history["train_loss"].append(train_results['loss'])
    history["train_acc"].append(train_results['accuracy'])
    history["val_loss"].append(val_results['loss'])
    history["val_acc"].append(val_results['accuracy'])

    if val_results['accuracy'] > best_acc:
        best_acc = val_results['accuracy']
        # best_state = {k:v.cpu() for k,v in model.state_dict().items()}
        bad = 0
    else:
        bad += 1
        if bad >= patience:
            print(f"Early stopping. Restauring state from epoch {epoch - bad}")
            break
        
print("Done!")

# if best_state is not None:
    # model.load_state_dict({k:v.to(device) for k,v in best_state.items()})


Starting training of LeaflictionCNN
-------------------------------
Epoch 1


100%|██████████| 111/111 [01:01<00:00,  1.80it/s]


train: train_loss=0.9941  train_acc=0.5491
val: val_loss=0.9483  val_acc=0.6158
-------------------------------
Epoch 2


100%|██████████| 111/111 [00:40<00:00,  2.72it/s]


train: train_loss=0.9127  train_acc=0.5868
val: val_loss=0.8868  val_acc=0.6337
-------------------------------
Epoch 3


...

train: train_loss=0.6339  train_acc=0.7353
val: val_loss=0.6016  val_acc=0.7624
-------------------------------
Epoch 9


100%|██████████| 111/111 [00:57<00:00,  1.94it/s]


train: train_loss=0.5677  train_acc=0.7698
val: val_loss=0.5950  val_acc=0.7584
-------------------------------
Epoch 10


100%|██████████| 111/111 [00:55<00:00,  1.99it/s]


train: train_loss=0.5625  train_acc=0.7682
val: val_loss=0.6038  val_acc=0.7426
Done!


In [7]:
from datetime import datetime

def plot_learning_curves(history, out_dir):
   
    filename = "learning_curves.png"

    Path(out_dir).mkdir(parents=True, exist_ok=True)

    epochs = range(1, len(history["train_loss"]) + 1)

    # --- Loss ---
    plt.figure()
    plt.plot(epochs, history["train_loss"], label="Train Loss")
    plt.plot(epochs, history["val_loss"],   label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Learning Curve – Loss")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.4)
    loss_path = Path(out_dir) / f"loss_{filename}"
    plt.savefig(loss_path, bbox_inches="tight", dpi=150)
    plt.close()

    # --- Accuracy ---
    plt.figure()
    plt.plot(epochs, history["train_acc"], label="Train Acc")
    plt.plot(epochs, history["val_acc"],   label="Val Acc")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Learning Curve – Accuracy")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.4)
    acc_path = Path(out_dir) / f"acc_{filename}"
    plt.savefig(acc_path, bbox_inches="tight", dpi=150)
    plt.close()

    print(f"Saved plots to:\n- {loss_path}\n- {acc_path}")




In [8]:
import csv
from pathlib import Path

def save_history_csv(history, out_dir):
    filename = "history.csv"
    out_path = Path(out_dir) / Path(filename)
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    with open(out_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["epoch", "train_loss", "train_acc", "val_loss", "val_acc"])
        for i in range(len(history["train_loss"])):
            writer.writerow([i+1,
                             history["train_loss"][i],
                             history["train_acc"][i],
                             history["val_loss"][i],
                             history["val_acc"][i]])
    print(f"Saved history to: {out_path}")




In [28]:
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

def export_classification_reports(
    train_results,
    val_results,
    out_dir,
    class_names=None,      # e.g. ["Apple Scab", "Black Rot", "Cedar Rust", "Healthy"]
    base_filename="classification_report",
    digits=3
):
    """
    train_results / val_results: dicts with 'y_true' and 'y_pred' (lists/ndarrays of ints)
    Exports:
      - plots/classification_report_train.txt
      - plots/classification_report_val.txt
      - plots/classification_report.csv  (rows=classes/averages, columns=precision/recall/f1/support per split)
    """
    Path(out_dir).mkdir(parents=True, exist_ok=True)

    y_true_train = np.asarray(train_results["y_true"])
    y_pred_train = np.asarray(train_results["y_pred"])
    y_true_val   = np.asarray(val_results["y_true"])
    y_pred_val   = np.asarray(val_results["y_pred"])

    # Consistent label order across splits
    labels = np.unique(np.concatenate([y_true_train, y_true_val]))
    target_names = class_names if (class_names is not None and len(class_names) == len(labels)) else None

    # -------- TXT (pretty) --------
    cr_train_txt = classification_report(
        y_true_train, y_pred_train, labels=labels, target_names=target_names, digits=digits, zero_division=0
    )
    cr_val_txt = classification_report(
        y_true_val, y_pred_val, labels=labels, target_names=target_names, digits=digits, zero_division=0
    )

    (Path(out_dir) / f"{base_filename}_train.txt").write_text(cr_train_txt)
    (Path(out_dir) / f"{base_filename}_val.txt").write_text(cr_val_txt)

    # -------- CSV (tidy) --------
    cr_train_dict = classification_report(
        y_true_train, y_pred_train, labels=labels, target_names=target_names, digits=digits, zero_division=0, output_dict=True
    )
    cr_val_dict = classification_report(
        y_true_val, y_pred_val, labels=labels, target_names=target_names, digits=digits, zero_division=0, output_dict=True
    )

    df_train = pd.DataFrame(cr_train_dict).T
    df_val   = pd.DataFrame(cr_val_dict).T

    # If keys are numeric strings, map to class_names (for readability)
    if target_names is not None:
        idx_map = {str(lbl): name for lbl, name in zip(labels, target_names)}
        df_train = df_train.rename(index=idx_map)
        df_val   = df_val.rename(index=idx_map)

    # Add split labels then concatenate
    df_train["split"] = "train"
    df_val["split"]   = "val"
    df_all = pd.concat([df_train, df_val], axis=0, ignore_index=False)

    out_csv = Path(out_dir) / f"{base_filename}.csv"
    df_all.to_csv(out_csv, index=True)

    print(f"Saved classification report to :\n- {Path(out_dir) / f'{base_filename}_train.txt'}\n- {Path(out_dir) / f'{base_filename}_val.txt'}\n- {out_csv}")


In [25]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def export_confusion_matrices(
    train_results, 
    val_results, 
    out_dir, 
    class_names=None,           # ex: ["Apple Scab", "Black Rot", "Cedar Rust", "Healthy"]
    normalize=None,             # None | "true" | "pred" | "all"
    filename="confusion_matrices.png"
):
    """
    train_results / val_results: dicts avec 'y_true' et 'y_pred' (list/ndarray d'entiers)
    normalize: None (comptes bruts) ou 'true'/'pred'/'all' pour des proportions
    Sauvegarde l'image dans out_dir/filename
    """
    y_true_train = np.asarray(train_results["y_true"])
    y_pred_train = np.asarray(train_results["y_pred"])
    y_true_val   = np.asarray(val_results["y_true"])
    y_pred_val   = np.asarray(val_results["y_pred"])

    # Déterminer les labels
    if class_names is None:
        labels = np.unique(np.concatenate([y_true_train, y_true_val]))
        display_labels = [str(x) for x in labels]
    else:
        labels = np.arange(len(class_names))
        display_labels = class_names

    # Confusion matrices (⚠️ train pour train, val pour val)
    cm_train = confusion_matrix(y_true_train, y_pred_train, labels=labels, normalize=normalize)
    cm_val   = confusion_matrix(y_true_val,   y_pred_val,   labels=labels, normalize=normalize)

    # Figure côte à côte
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    fmt = ".2f" if normalize else "d"

    disp_train = ConfusionMatrixDisplay(cm_train, display_labels=display_labels)
    disp_val   = ConfusionMatrixDisplay(cm_val,   display_labels=display_labels)

    disp_train.plot(ax=axes[0], values_format=fmt)
    disp_val.plot(ax=axes[1],   values_format=fmt)

    axes[0].set_title("Train")
    axes[1].set_title("Validation")
    fig.suptitle("Confusion Matrices", y=0.98)
    fig.tight_layout()

    Path(out_dir).mkdir(parents=True, exist_ok=True)
    out_path = Path(out_dir) / filename
    fig.savefig(out_path, dpi=150, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved confusion matrices to {out_path}")


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(val_results['y_true'], val_results['y_pred'], digits=3))
print(classification_report(train_results['y_true'], train_results['y_pred'], digits=3))

              precision    recall  f1-score   support

           0      0.514     0.899     0.654        99
           1      0.960     0.733     0.831       262
           2      0.843     0.977     0.905        44
           3      0.630     0.510     0.564       100

    accuracy                          0.743       505
   macro avg      0.737     0.780     0.739       505
weighted avg      0.797     0.743     0.750       505

              precision    recall  f1-score   support

           0      0.624     0.757     0.684      3416
           1      0.873     0.755     0.810      3577
           2      0.852     0.994     0.918      3601
           3      0.740     0.565     0.641      3595

    accuracy                          0.768     14189
   macro avg      0.772     0.768     0.763     14189
weighted avg      0.774     0.768     0.764     14189



In [ ]:
out_dir = "learning_curves_" + str(datetime.now())

In [29]:

plot_learning_curves(history, out_dir)
save_history_csv(history, out_dir)
export_confusion_matrices(train_results, val_results, out_dir)
export_classification_reports(train_results, val_results, out_dir=out_dir)

Saved plots to:
- learning_curves_2025-09-17 18:27:06.952330/loss_learning_curves.png
- learning_curves_2025-09-17 18:27:06.952330/acc_learning_curves.png
Saved history to: learning_curves_2025-09-17 18:27:06.952330/history.csv
Saved confusion matrices to learning_curves_2025-09-17 18:27:06.952330/confusion_matrices.png
Saved classification report to :
- learning_curves_2025-09-17 18:27:06.952330/classification_report_train.txt
- learning_curves_2025-09-17 18:27:06.952330/classification_report_val.txt
- learning_curves_2025-09-17 18:27:06.952330/classification_report.csv
